In [3]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 100)

In [4]:
discover_data = pd.read_csv("discover_user_input_results.csv")
discover_data.head()

,Unnamed: 0,email,sensitivity,skinType,timestamp,concern_0,concern_1,concern_2,concern_3,concern_4,concern_5,concern_6,concern_7,concern_8,concern_9,concern_10,concern_11,concern_12,concern_13,result_0,result_1,result_2,result_3,result_4,result_5,result_6,result_7,result_8,result_9
0,935,jeremy@paulaschoice.sg,False,Combination,"September 9th 2017, 1:56:08 pm",Dehydration,Sun Damage,Men,Combination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7830,7780,7660,7740,7860,1720,8740,2760,5800,7880
1,936,jeremy@paulaschoice.sg,False,Oily,"September 9th 2017, 1:58:21 pm",Acne,Wrinkles,PIH,Men,Oily,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1150,7670,8720,7740,7870,6130,8740,2760,5700,6240
2,937,ck1411@singnet.com.sg,False,Combination,"September 9th 2017, 2:38:56 pm",Enlarged Pores,Sun Damage,Wrinkles,Dehydration,Loss of Firmness,Dullness,Combination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7830,7780,7820,7740,7870,7760,7690,2760,7900,7960
3,938,jeremy@paulaschoice.sg,False,Oily,"September 9th 2017, 2:51:12 pm",Clogged Pores,Redness,Uneven Texture,Enlarged Pores,Acne,Wrinkles,Dehydration,Sun Damage,PIH,Dullness,Loss of Firmness,Men,Oily,NaN,7830,7670,8720,7740,7870,7800,8740,2760,5700,7730
4,939,starlites18@gmail.com,False,Combination,"September 9th 2017, 2:59:07 pm",Enlarged Pores,Clogged Pores,Acne,Sun Damage,Uneven Texture,Redness,Combination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6002,1350,2010,7740,7870,6130,7690,2750,5700,7730


In [27]:
discover_first = discover_data.drop_duplicates(subset = ['email'], keep='first')
discover_first['timestamp2'] = pd.to_datetime(discover_first['timestamp'].map(lambda x: x.replace('th', "")))
discover_first.drop('timestamp', axis=1,inplace=True)
discover_first.to_csv('discover_first.csv')

C:\Users\limzi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\limzi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
sales_data = pd.read_csv("shopify_orders_export_20180207.csv", 
                         low_memory=False, 
                         parse_dates=['Paid at', 'Fulfilled at', 'Created at'])

sales_data_clean = sales_data.drop(sales_data.columns.to_series()[-11:-1], axis=1)
sales_data_clean.dropna(subset=['Email'], axis=0, inplace=True)

In [7]:
sales_data_clean['discover_first_date'] = sales_data_clean['Email'].map(discover_first.set_index('email')['timestamp2'])
sales_data_clean['used_discover_already'] = (sales_data_clean['Created at']> sales_data_clean['discover_first_date']).map({True: "Used Discover", False: "Not yet"})
sales_data_clean['discover_sales_lead_time'] = sales_data_clean['Created at'] - sales_data_clean['discover_first_date']

In [8]:
pre_discover_sales = sales_data_clean[sales_data_clean['Created at']< "2017-09-09"]
post_discover_sales = sales_data_clean[sales_data_clean['Created at']>= "2017-09-09"]

In [9]:
'''
comparison_table = sales_data_clean[['Email', 'Created at', 'discover_first_date', 'used_discover_already']]
comparison_table['discover_sales_lead_time'] = comparison_table['Created at'] - comparison_table['discover_first_date']
'''
same_day_purchase = sales_data_clean[(sales_data_clean['discover_sales_lead_time'] <= "1 days") & (sales_data_clean['discover_sales_lead_time'] > "0 days")]
same_day_purchases_count = len(same_day_purchase['Email'].unique())
post_discover_launch_customer_count = len(discover_first['email'].unique())
percentage_same_day_purchase = same_day_purchases_count/post_discover_launch_customer_count
print("same_day_purchases_count:", same_day_purchases_count)
print("post_discover_launch_customer_count:", post_discover_launch_customer_count)
print("percentage_same_day_purchase:", percentage_same_day_purchase)
# same_day_purchase.to_excel("same_day_purchase.xlsx")

same_day_purchases_count: 335
post_discover_launch_customer_count: 2422
percentage_same_day_purchase: 0.13831544178364988


### Following code aims to "unpivot" ("melt" in Pandas term) in discover_first[result_SKUs].
### The objective is to allow for easier indexing to search for SKUs in sales_data_clean

In [10]:
# pivot_discover_first = discover_first.unstack(index='email', columns = ['result_0', 'result_1',
#       'result_2', 'result_3', 'result_4', 'result_5', 'result_6', 'result_7',
#       'result_8', 'result_9'])
pivot_discover_first = pd.melt(discover_first, 
                               id_vars='email',
                               value_vars=list(discover_first.columns[-11:-1]),
                               var_name='step_no',
                               value_name= 'SKU')

In [23]:
pivot_discover_first.sort_values(['email', 'step_no'], inplace=True)
# pivot_discover_first.to_excel("pivot_discover_first.xlsx")

In [26]:
pivot_discover_first

,email,step_no,SKU
2382,007lavender@gmail.com,result_0,6002
4804,007lavender@gmail.com,result_1,7670
7226,007lavender@gmail.com,result_2,6200
9648,007lavender@gmail.com,result_3,7740
12070,007lavender@gmail.com,result_4,7870
14492,007lavender@gmail.com,result_5,6130
16914,007lavender@gmail.com,result_6,7690
19336,007lavender@gmail.com,result_7,2750
21758,007lavender@gmail.com,result_8,5700
24180,007lavender@gmail.com,result_9,6240


In [25]:
pivot_discover_first['PurchaseCount'] = same_day_purchase['Email'].map(pivot_discover_first.set_index('email')['SKU'])
pivot_discover_first

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [12]:
discover_first.columns[-10:-1]

Index(['result_1', 'result_2', 'result_3', 'result_4', 'result_5', 'result_6',
       'result_7', 'result_8', 'result_9'],
      dtype='object')

### Analyze users who have made a purchase within a day of using Discover (for the first time)

In [13]:
discover_first[discover_first['email'] == "yuleandra21@gmail.com"]

,Unnamed: 0,email,sensitivity,skinType,concern_0,concern_1,concern_2,concern_3,concern_4,concern_5,concern_6,concern_7,concern_8,concern_9,concern_10,concern_11,concern_12,concern_13,result_0,result_1,result_2,result_3,result_4,result_5,result_6,result_7,result_8,result_9,timestamp2
1000,1935,yuleandra21@gmail.com,False,Oily,Acne,PIH,Oily,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1150,7670,2040,7740,7770,6130,3400,2760,5700,6240,2017-10-05 03:47:31


In [14]:
# same_day_purchase.groupby(by=['Email','Lineitem sku'])['Lineitem sku'].count()
pivot_discover_first['Purchased?'] = pivot_discover_first['email'].map(same_day_purchase.set_index(['Email', 'Lineitem sku'])[])

SyntaxError: invalid syntax (<ipython-input-14-e8bb11a6b750>, line 2)

### Faced issues creating a frequency plot for SKUs under each email based on sales data
### Abandoned using jupyter, tried on excel instead, also gave up subsequently
(Might need to use for loop with two filter criteria, but could not figure out how)

### Analyze post-discover-launch sales data of only users who have used Discover

In [ ]:
only_users_from_post_discover_sales = post_discover_sales[post_discover_sales['Email'].isin(discover_first['email'].unique())]
only_users_from_post_discover_sales

## How long does it take for brand new customers to buy something after using Discover?

### First, identify customers who had not purchased before Discover launch

In [15]:
new_customers_test = pd.DataFrame(post_discover_sales['Email'].unique(), columns=["Post Launch Emails"])
new_customers_test['Exist before launch?'] = new_customers_test.isin(pre_discover_sales['Email'].unique())
# new_customers_test['First Tried Discover'] = 
new_customers_test = new_customers_test.merge(right=discover_first[['email', 'timestamp2']], left_on='Post Launch Emails', right_on='email')
new_customers_test = new_customers_test.drop('email', axis=1)
new_customers_test = new_customers_test.rename(index=str, columns={"timestamp2": "First Tried Discover"})

In [16]:
# get first transaction date of each of the "new" customers
post_discover_first_purchase = post_discover_sales[['Email', 'Created at']].drop_duplicates(subset='Email', keep='last')
print("Number of rows:", len(post_discover_first_purchase))
post_discover_first_purchase

Number of rows: 2847


,Email,Created at
4,gilly.glanville@me.com,2018-02-08 03:59:00
9,alyssacmy@gmail.com,2018-02-07 13:45:51
11,jglyj82@gmail.com,2018-02-07 13:02:33
18,wailing_93@hotmail.com,2018-02-07 11:25:32
21,jesse.dytioco@gmail.com,2018-02-07 11:20:49
25,veron_chinron@hotmail.com,2018-02-07 10:09:37
31,weizen.t@gmail.com,2018-02-07 08:02:53
32,lady_portia3012@yahoo.com.sg,2018-02-07 07:43:11
41,nisha@vstravel.com.sg,2018-02-07 05:24:51
49,hhshelley1@gmail.com,2018-02-07 04:00:22


In [17]:
new_customers_test = new_customers_test.merge(post_discover_first_purchase, left_on="Post Launch Emails", right_on="Email")
new_customers_test = new_customers_test.drop('Email', axis=1).rename(index=str, columns={"Created at": "First Purchase Date"})

In [18]:
# To obtain the list of unique emails of NEW customers who did not take Discover
pikachu = pd.DataFrame(post_discover_sales['Email'].unique(), columns=["Post Launch Emails"])
new_customers_no_discover = pikachu[~pikachu.isin(pre_discover_sales['Email'].unique())]
new_customers_no_discover = new_customers_no_discover.dropna()
new_customers_no_discover

,Post Launch Emails
0,gilly.glanville@me.com
1,stephaniedata@yahoo.com
2,hsmeaton@hotmail.com
4,alyssacmy@gmail.com
5,jglyj82@gmail.com
8,wailing_93@hotmail.com
9,jesse.dytioco@gmail.com
14,weizen.t@gmail.com
18,serene818@hotmail.com
20,boazruth76@yahoo.com


In [19]:
new_customers_test['Time To Buy'] = new_customers_test['First Purchase Date'] - new_customers_test['First Tried Discover']
new_customers_test = new_customers_test[(new_customers_test['Time To Buy']> "0 days") & (new_customers_test['Exist before launch?'] == False)]

## Lead time to purchase for new customers after taking Discover

In [20]:
print(new_customers_test['Time To Buy'].describe())
print(new_customers_test['Time To Buy'].quantile(0.85))


count                        346
mean     10 days 03:05:21.540462
std      21 days 16:32:44.176929
min              0 days 00:01:22
25%       0 days 00:19:19.500000
50%       0 days 09:24:47.500000
75%       9 days 00:29:25.250000
max            139 days 05:01:07
Name: Time To Buy, dtype: object
22 days 01:29:42.500000


import matplotlib.pyplot as plt
plt.boxplot(new_customers_test['Time To Buy'])
plt.show()

## Average Order Value of _New_ Customers who made their first purchase after taking Discover

In [21]:
new_discover_sale_transaction = post_discover_sales[post_discover_sales['Email'].isin(new_customers_test['Post Launch Emails'].unique())]
# new_discover_sale_transaction.groupby(by=['Email', 'Created at'])['Total'].sum()
new_discover_sale_transaction['Total'].mean()

92.17494824016563

## Average Order Value of _New_ Customers who made their first purchase _without_ taking Discover

In [22]:
new_non_discover_sale_transaction = post_discover_sales[~post_discover_sales['Email'].isin(new_customers_test['Post Launch Emails'].unique())]
new_non_discover_sale_transaction['Total'].mean()

105.15647869674186